## ВКР

Автоматизация подбора персонала

Копчев Владислав, БПМИ197

### Библиотеки

selenium поскольку нужен js

In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# !pip install transliterate
from transliterate import translit

### Скрейпинг страницы со списком резюме

hh.ru не принимает названия с заглавными буквами:

In [78]:
def normalize_translit(ru_query):
    text = translit(ru_query, language_code='ru', reversed=True)
    text = text.replace(' ', '-').replace('yj', 'yy').replace('yy-', 'yy_').replace('es-', 'es_').replace('k-pr', 'k_pr')  # эвристическое правило, не знаю, почему у hh так...
    return text

In [82]:
driver = webdriver.Chrome('./chromedriver')

def scrape(query, driver):
    pages_count = 10  # соскрейпить это число иначе ошибки!
    for n in range(pages_count):  # проблема!!
        link = 'https://hh.ru/resumes/{}?items_on_page=100&page={}'.format(query, n)  # 100 per page
        driver.get(link)

        resumes_list = driver.find_element(by=By.XPATH, value='//div[@class="resume-search-item__header"]')
        # resumes_list.click()  # стоп а зачем это...
        resumes = resumes_list.find_element(by=By.XPATH, value='//div[@data-qa="resume-serp__results-search"]').find_elements(by=By.XPATH, value='//a[@class="serp-item__title"]')

        resumes_parsed = []
        for x in resumes:
            resumes_parsed.append((x.text, x.get_attribute("href")))

        if n == 0:
            df = pd.DataFrame(resumes_parsed, columns=['Вакансия', 'Ссылка'])
            # print(len(resumes_parsed))
        else:
            # print(len(resumes_parsed))
            df2 = pd.DataFrame(resumes_parsed, columns=['Вакансия', 'Ссылка'])  # , index=list(range(0 + 100 * n, 100 + 100 * n)) n00, ..., n99n = 0, ..., 49
            df = pd.concat([df, df2])
    
    return df

<ipython-input-82-c83a548329b0>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [83]:
analysts_list = ['аналитик bi',
                'системный аналитик',
                'бизнес аналитик',
                'аналитик продаж',
                'финансовый аналитик',
                'аналитик данных',
                'data analyst']
analysts_map = dict()

for analyst in analysts_list:
    analysts_map[analyst] = scrape(normalize_translit(analyst), driver)
    analysts_map[analyst].index = range(analysts_map[analyst].shape[0])

Конкатеннация результатов:

In [85]:
df1 = pd.DataFrame()

for analyst in analysts_list:
    df2 = analysts_map[analyst]
    df1 = pd.concat([df1, df2])
    
df1.index = range(df1.shape[0])
df1.to_csv('resumes_all.csv')

### Скрейпинг страницы с конкретным резюме

In [93]:
def normalize(x):
    if x:
        x = x[0].text
    else:
        x = '---'  # так будет лучше чем просто ' '?
    return x

In [94]:
def scrape(link, driver):
    driver.get(link)
    comandirovka = driver.find_elements(by=By.XPATH, value='//div[@class="bloko-translate-guard"]')
    comandirovka = normalize(comandirovka)  # норм?
    
    opyt = driver.find_elements(by=By.XPATH, value='//span[@class="resume-block__title-text resume-block__title-text_sub"]')
    opyt = normalize(opyt)

    about = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block-container"][@data-qa="resume-block-skills-content"]')
    about = normalize(about)
        
    educ = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block"][@data-qa="resume-block-education"]')
    educ = normalize(educ)
        
    inter = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block-item-gap"]')
    inter = normalize(inter)
        
    inter2 = driver.find_elements(by=By.XPATH, value='//div[@class="key-skills-row"][@data-qa="tags-key-skills"]')
    inter2 = normalize(inter2)
        
    return (comandirovka, opyt, about, educ, inter, inter2, link)

In [97]:
df_links = list(pd.read_csv('resumes_all.csv')['Ссылка'])[:100]

resume_list = []
driver = webdriver.Chrome('./chromedriver')
for link in df_links:
    # print(link)
    data = scrape(link, driver)
    resume_list.append(data)

df = pd.DataFrame(resume_list, columns=['Коммандировка', 
                                        'Опыт',
                                        'О себе',
                                        'Образование',
                                        'Интересы',
                                        'Навыки',
                                        'Ссылка'])
df.to_csv('resumes_features.csv')
df.head(2)

<ipython-input-97-68cf740f9a54>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


,Коммандировка,Опыт,О себе,Образование,Интересы,Навыки,Ссылка
0,"Moscow, willing to relocate, prepared for occa...",Work experience 7 years 4 months,"Responsible, communicable, quick study and det...",Higher education\n2015\nMOSCOW STATE UNIVERSIT...,"Specializations:\nSales manager, account manag...",Поддержка продаж / владение ПО для поддержки п...,https://hh.ru/resume/1aa91e410000fd3cda0039ed1...
1,"Санкт-Петербург, м. Гражданский проспект, гото...",Опыт работы 7 лет 2 месяца,Имею экономическое образование. Продвинутый по...,Высшее образование (Бакалавр)\n2015\nСанкт-Пет...,Специализации:\nАналитик\nЗанятость: полная за...,"Работа с компьютерными программами, программы ...",https://hh.ru/resume/bebf6a630008069d8c0039ed1...


^ скрейпер работает ок, буду собирать только 100 вакансий, сейчас учу его собирать новые признаки. дальше бекапнуть resumes_features!!

Открыть рандом вакансию и соскрейпить это

```
'Ready_2_move' - флаг ‘готов к переезду’
'Ready_4_business_trip' -  флаг ‘готов к командировкам’
'Description' - личное описание кандидата 
'Tags' - теги (основные навыки и интересы)
'Graduation' - образование 
'EduLevel' - уровень образования 
'LastUni' - последний университет 1
'LastUni_2' - последний университет 2
'N_Unis' - количество мест, где человек обучался
'URL'
```

Что теперь?

1. Научить его выгружать не список образований/работ текстом, а списком. После этого создать фичи с последними двумя местами работы. А также двумя университетами и кол-вом образований. 

2. Научить его собирать Ссылку и Имя, притом в начале таблицы

3. Начать делать справочник по образованию (различные варианты обозначить текстом один и тот же вуз) и добавить признак топовости образования